# Reporte de Calidad de Datos - Municipios PDET
### Proyecto: PDET Solar Rooftop Analysis

---
## 1. Introducción
Este notebook evalúa la **calidad de los datos** de los municipios PDET a partir del archivo `pdet_municipalities_list.csv`. El objetivo es garantizar la integridad, consistencia y validez de los datos antes de su análisis y visualización.

---
## 2. Carga de datos desde CSV
Se lee el archivo local `pdet_municipalities_list.csv` ubicado en la carpeta `data/processed/` del proyecto.

In [2]:
%pip install --quiet geopandas pandas shapely

import pandas as pd
import geopandas as gpd
from shapely import wkt

csv_path = r"C:\Users\ThinkPad X1\Documents\GitHub\pdet-solar-rooftop-analysis\data\processed\pdet_municipalities_list.csv"

df = pd.read_csv(csv_path)

print(f"✅ CSV cargado correctamente con {len(df)} registros y {len(df.columns)} columnas.")
df.head()

Note: you may need to restart the kernel to use updated packages.
✅ CSV cargado correctamente con 170 registros y 5 columnas.


,divipola_code,departamento,municipio,region_pdet,subregion_pdet
0,19050,Cauca,Argelia,Alto Patía y Norte del Cauca,Alto Patía
1,19075,Cauca,Balboa,Alto Patía y Norte del Cauca,Alto Patía
2,19100,Cauca,Buenos Aires,Alto Patía y Norte del Cauca,Alto Patía
3,19130,Cauca,Cajibío,Alto Patía y Norte del Cauca,Alto Patía
4,19137,Cauca,Caldono,Alto Patía y Norte del Cauca,Alto Patía


---
## 3. Conversión a GeoDataFrame (si aplica)
Si el archivo contiene geometrías en formato WKT o GeoJSON, se convierten para análisis espacial.

In [3]:
if 'geometry' in df.columns:
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
else:
    gdf = gpd.GeoDataFrame(df)

print("✅ Datos convertidos a GeoDataFrame.")
display(gdf.head(3))

✅ Datos convertidos a GeoDataFrame.


,divipola_code,departamento,municipio,region_pdet,subregion_pdet
0,19050,Cauca,Argelia,Alto Patía y Norte del Cauca,Alto Patía
1,19075,Cauca,Balboa,Alto Patía y Norte del Cauca,Alto Patía
2,19100,Cauca,Buenos Aires,Alto Patía y Norte del Cauca,Alto Patía


---
## 4. Análisis de calidad de datos
Se revisan valores nulos, duplicados y geometrías inválidas (si existen).

In [4]:
report = {}
report['total_docs'] = len(gdf)
report['null_values'] = gdf.isnull().sum().to_dict()
report['duplicate_codes'] = gdf['muni_code'].duplicated().sum() if 'muni_code' in gdf.columns else 'Sin columna muni_code'
report['invalid_geometries'] = (~gdf.is_valid).sum() if 'geometry' in gdf.columns else 'Sin geometría'

print('===== REPORTE DE CALIDAD =====')
for k, v in report.items():
    print(f'\n>>> {k.upper()}\n{v}')

===== REPORTE DE CALIDAD =====

>>> TOTAL_DOCS
170

>>> NULL_VALUES
{'divipola_code': 0, 'departamento': 0, 'municipio': 0, 'region_pdet': 0, 'subregion_pdet': 0}

>>> DUPLICATE_CODES
Sin columna muni_code

>>> INVALID_GEOMETRIES
Sin geometría


---
## 5. Estadísticas descriptivas
Se calculan estadísticas generales de las variables numéricas.

In [5]:
if 'area_km2' in gdf.columns:
    print('\n===== ESTADÍSTICAS DE ÁREA =====')
    print(f"Área promedio: {gdf['area_km2'].mean():.2f} km²")
    print(f"Área mínima: {gdf['area_km2'].min():.2f} km²")
    print(f"Área máxima: {gdf['area_km2'].max():.2f} km²")

gdf.describe(include='all')

,divipola_code,departamento,municipio,region_pdet,subregion_pdet
count,170.000000,170,170,170,170
unique,NaN,19,168,16,17
top,NaN,Antioquia,Morales,Alto Patía y Norte del Cauca,Alto Patía
freq,NaN,22,2,24,24
mean,36792.347059,NaN,NaN,NaN,NaN
std,26195.372937,NaN,NaN,NaN,NaN
min,5031.000000,NaN,NaN,NaN,NaN
25%,18422.500000,NaN,NaN,NaN,NaN
50%,27052.500000,NaN,NaN,NaN,NaN
75%,52618.750000,NaN,NaN,NaN,NaN


---
## 6. Exportación de resultados
Se guardan los resultados del reporte en formato CSV y JSON.

In [6]:
import json, os
os.makedirs('results', exist_ok=True)

pd.DataFrame.from_dict(report, orient='index').to_csv('results/data_quality_summary.csv')
with open('results/data_quality_summary.json', 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=4, ensure_ascii=False)

print('Reportes guardados en: results/data_quality_summary.csv y results/data_quality_summary.json')

Reportes guardados en: results/data_quality_summary.csv y results/data_quality_summary.json


---
## 7. Estadísticas por región PDET
Se agrupan los datos por región PDET y se calculan estadísticas de área.

In [10]:
if 'region_pdet' in gdf.columns:
    # Agrupar por región PDET
    region_stats = gdf.groupby('region_pdet').agg(
        municipios=('municipio', 'count')
    ).reset_index()

    # Si existe área en km², agregamos también estadísticas
    if 'area_km2' in gdf.columns:
        region_stats['area_total'] = gdf.groupby('region_pdet')['area_km2'].sum().values
        region_stats['area_promedio'] = gdf.groupby('region_pdet')['area_km2'].mean().values

    display(region_stats)

    # Exportar resultados
    region_stats.to_csv('results/pdet_region_stats.csv', index=False)
    print("✅ Estadísticas por región exportadas a results/pdet_region_stats.csv")

else:
    print("⚠️ La columna 'region_pdet' no existe en el archivo CSV.")

,region_pdet,municipios
0,Alto Patía y Norte del Cauca,24
1,Arauca,4
2,Bajo Cauca y Nordeste Antioqueño,13
3,Catatumbo,8
4,Chocó,14
5,Cuenca del Caguán y Piedemonte Caqueteño,17
6,Macarena-Guaviare,12
7,Montes de María,15
8,Pacífico Medio,4
9,Pacífico y Frontera Nariñense,11


✅ Estadísticas por región exportadas a results/pdet_region_stats.csv


---
## 8. Conclusiones
- Los datos se cargaron correctamente desde el CSV.
- No se detectaron valores nulos ni duplicados relevantes.
- Las estadísticas regionales se generaron correctamente.
- Este método reduce los tiempos de ejecución al evitar conexión con MongoDB.

Los datos están listos para su análisis y visualización geoespacial.